---
title: "Quick Poll: Economic Concepts"
format: html
sidebar: microeconomics
jupyter: python3
---



::: {.poll-form}
# Quick Economics Poll

**Question:** What best describes the relationship between price and demand?

<form id="pollForm">
  <div class="form-group">
    <input type="text" id="studentName" placeholder="Your name (for attendance)" required style="width: 100%; padding: 0.5rem; margin-bottom: 1rem;">
  </div>
  
  <div class="form-group">
    <label>
      <input type="radio" name="answer" value="inverse" required> 
      **Inverse relationship** - As price increases, demand decreases
    </label>
  </div>
  
  <div class="form-group">
    <label>
      <input type="radio" name="answer" value="direct" required>
      **Direct relationship** - As price increases, demand increases  
    </label>
  </div>
  
  <div class="form-group">
    <label>
      <input type="radio" name="answer" value="no-relationship" required>
      **No relationship** - Price and demand are independent
    </label>
  </div>
  
  <button type="submit" class="btn-poll">Submit Response</button>
</form>
:::

::: {.response-display}
### Live Results
<div id="results">Loading results...</div>
<button onclick="loadResults()" class="btn-poll" style="margin-top: 1rem;">Refresh Results</button>
:::

<script>
// Configuration
const GITHUB_TOKEN = 'your_github_token'; // Store in GitHub Secrets for production
const REPO_OWNER = 'your-username';
const REPO_NAME = 'your-repo-name';
const POLL_ISSUE_NUMBER = 1; // Create an issue for this poll

// Submit poll response
document.getElementById('pollForm').addEventListener('submit', async function(e) {
    e.preventDefault();
    
    const name = document.getElementById('studentName').value;
    const answer = document.querySelector('input[name="answer"]:checked').value;
    
    const comment = `**Student:** ${name}
**Answer:** ${answer}
**Timestamp:** ${new Date().toISOString()}`;
    
    try {
        const response = await fetch(`https://api.github.com/repos/${REPO_OWNER}/${REPO_NAME}/issues/${POLL_ISSUE_NUMBER}/comments`, {
            method: 'POST',
            headers: {
                'Authorization': `token ${GITHUB_TOKEN}`,
                'Accept': 'application/vnd.github.v3+json',
                'Content-Type': 'application/json'
            },
            body: JSON.stringify({
                body: comment
            })
        });
        
        if (response.ok) {
            alert('Response submitted successfully!');
            document.getElementById('pollForm').reset();
            loadResults();
        } else {
            alert('Error submitting response. Please try again.');
        }
    } catch (error) {
        console.error('Error:', error);
        alert('Error submitting response. Please check your connection.');
    }
});

// Load and display results
async function loadResults() {
    try {
        const response = await fetch(`https://api.github.com/repos/${REPO_OWNER}/${REPO_NAME}/issues/${POLL_ISSUE_NUMBER}/comments`);
        const comments = await response.json();
        
        // Parse results
        const results = { inverse: 0, direct: 0, 'no-relationship': 0 };
        const students = new Set();
        
        comments.forEach(comment => {
            const lines = comment.body.split('\n');
            const studentLine = lines.find(line => line.startsWith('**Student:**'));
            const answerLine = lines.find(line => line.startsWith('**Answer:**'));
            
            if (studentLine && answerLine) {
                const student = studentLine.replace('**Student:**', '').trim();
                const answer = answerLine.replace('**Answer:**', '').trim();
                
                // Only count each student once (prevent duplicate votes)
                if (!students.has(student)) {
                    students.add(student);
                    if (results.hasOwnProperty(answer)) {
                        results[answer]++;
                    }
                }
            }
        });
        
        // Display results
        const total = students.size;
        let html = `<h4>Results (${total} responses)</h4>`;
        
        const labels = {
            'inverse': 'Inverse relationship (correct)',
            'direct': 'Direct relationship', 
            'no-relationship': 'No relationship'
        };
        
        Object.entries(results).forEach(([answer, count]) => {
            const percentage = total > 0 ? Math.round((count / total) * 100) : 0;
            html += `
                <div style="margin: 0.5rem 0; display: flex; align-items: center;">
                    <div style="width: 200px; font-weight: 500;">${labels[answer]}:</div>
                    <div style="width: 100px; padding: 0.25rem 0.5rem; background: #007bff; color: white; text-align: center; border-radius: 0.25rem; margin-right: 1rem;">
                        ${count} (${percentage}%)
                    </div>
                    <div style="flex: 1; height: 20px; background: #e9ecef; border-radius: 10px; overflow: hidden;">
                        <div style="width: ${percentage}%; height: 100%; background: #007bff;"></div>
                    </div>
                </div>
            `;
        });
        
        document.getElementById('results').innerHTML = html;
        
    } catch (error) {
        console.error('Error loading results:', error);
        document.getElementById('results').innerHTML = 'Error loading results. Please try again.';
    }
}

// Load results when page loads
document.addEventListener('DOMContentLoaded', loadResults);
</script>

---

**Instructions for Instructor:**
1. Create a GitHub issue in your repository titled "Poll: Economic Concepts"
2. Note the issue number and update the `POLL_ISSUE_NUMBER` variable
3. Create a Personal Access Token with repo permissions  
4. Store the token securely (use GitHub Secrets for production)
5. Students' responses will appear as comments on the issue